# Facial Expressions Recognition

## Introduction

By [Rifky Aliffa](https://github.com/Penzragon)

![Image](https://www.duarte.com/wp-content/uploads/Facial-Expression_1.png)

### Dataset

The dataset used in this project is the fer2013 dataset. The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centred and occupies about the same amount of space in each image. You can download the dataset from [Kaggle](https://www.kaggle.com/datasets/msambare/fer2013).

### Objective

The objective of this project is to build a convolutional neural network model that can recognize human facial expressions.

## Import Libraries

Pada project ini library utama yang digunakan adalah **Pandas**, **NumPy**, **Matplotlib**, **Seaborn**, **Scikit-Learn**, **Tensorflow**, dan **Keras**.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

## Data Loading

Pada bagian ini data gambar akan di load menggunakan `ImageDataGenerator`.

In [3]:
train_path = 'dataset/train'
test_path = 'dataset/test'

In [4]:
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, horizontal_flip=True, vertical_flip=True, validation_split=0.2)
test_gen = ImageDataGenerator(rescale=1./255)

In [5]:
train_data = train_gen.flow_from_directory(train_path, target_size=(48, 48), batch_size=23, class_mode='categorical', color_mode='grayscale', subset='training')
valid_data = train_gen.flow_from_directory(train_path, target_size=(48, 48), batch_size=23, class_mode='categorical', color_mode='grayscale', subset='validation')
test_data = test_gen.flow_from_directory(test_path, target_size=(48, 48), batch_size=23, class_mode='categorical', color_mode='grayscale', shuffle=False)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
classes = list(train_data.class_indices.keys())
print(f'Classes: {classes}')

Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


Pada dataset ini terdapat **7 kelas** yang berbeda, yaitu:
- Angry
- Disgust
- Fear
- Happy
- Neutral
- Sad
- Surprise